Exam if January Effect exist in S&P500 stocks 
Data source:
https://www.kaggle.com/camnugent/sandp500/downloads/sandp500.zip/4
    

In [2]:
import pandas as pd
import sklearn as sk

In [3]:
fname = 'all_stocks_5yr.csv'

In [4]:
data = pd.read_csv(fname, parse_dates=['date'], infer_datetime_format=True)

In [5]:
data.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [6]:
comps = data['Name'].drop_duplicates()
comp_num = comps.shape[0]
print(f"Total number of companies {comp_num}")

Total number of companies 505


In [7]:
dates = data['date'].map(lambda x: x.strftime('%Y') + '-' + x.strftime('%m')).drop_duplicates().values
print(f"start date = {dates.min()} \n"
      f"end date = {dates.max()}")

start date = 2013-02 
end date = 2018-02


In [29]:
def calculate_p(result):
    val = result.values()
    pos = [x for x in val if x > 0]
    percent = len(pos)/len(val)*100
    return percent

Exam the average price in January avg_jan, compared to Average Price in December avg_dec 


In [8]:
avg_p_result = {}
yrs = range(2014, 2018)
for c in comps:
    df = data[data['Name'] == c]
    for y in yrs:
        jan = df[ (df['date'] >= "{}-01-01".format(y)) & (df['date'] < "{}-02-01".format(y))]
        avg_jan = jan[['open', 'close']].mean()
        
        dec = df[ (df['date'] >= "{}-12-01".format(y-1)) & (df['date'] < "{}-01-01".format(y))]
        avg_dec = dec[['open', 'close']].mean()
        
        comp =  (avg_jan > avg_dec)
        score = comp.sum()
        avg_p_result[c] = score

In [30]:
avg_p = calculate_p(avg_p_result)
avg_p

65.94059405940594

Looks like it is only true in 66% of the time

What if we narrow down the criteria and exam the first opening of the year compared to last close?

In [25]:
open_close_result = {}
for c in comps:
    df = data[data['Name'] == c]
    for y in yrs:
        jan = df[ (df['date'] >= "{}-01-01".format(y)) & (df['date'] < "{}-02-01".format(y))].head(1)
        jan_open = jan['open']
        
        dec = df[ (df['date'] >= "{}-12-01".format(y-1)) & (df['date'] < "{}-01-01".format(y))].head(1)
        dec_close = dec['close']
        
        comp =  jan_open.values > dec_close.values
        score = comp.sum()
        open_close_result[c] = score

In [31]:
open_close_p = calculate_p(open_close_result)
open_close_p

70.6930693069307

Instead of comparing the open/clost price, let's compare the profit. Assuming purchase on the 1st day of the month at open and sell on the last day at close

In [67]:
profit_result = {}
for c in comps:
    df = data[data['Name'] == c]
    profit_result[c] = 0
    for y in yrs:
        jan_purchase = df[ (df['date'] >= "{}-01-01".format(y)) & (df['date'] < "{}-02-01".format(y))].head(1)
        jan_sell = df[ (df['date'] >= "{}-01-01".format(y)) & (df['date'] < "{}-02-01".format(y))].tail(1)
        jan_profit = jan_purchase['open'].values - jan_sell['close'].values
                
        dec_purchase = df[ (df['date'] >= "{}-12-01".format(y-1)) & (df['date'] < "{}-01-01".format(y))].head(1)
        dec_sell = df[ (df['date'] >= "{}-12-01".format(y-1)) & (df['date'] < "{}-01-01".format(y))].tail(1)
        dec_profit = dec_purchase['open'].values - dec_sell['close'].values
        
        comp =  jan_profit > dec_profit
        score = comp.sum()
        profit_result[c] += score
    profit_result[c] = profit_result[c] * 1.0/ len(yrs) 

In [69]:
profit_p = sum(profit_result.values()) / len(comps) * 100.0
profit_p

57.67326732673267

Hmmm how about the first week?

In [70]:
profit_1w_result = {}
for c in comps:
    df = data[data['Name'] == c]
    profit_1w_result[c] = 0
    for y in yrs:
        jan_purchase = df[ (df['date'] >= "{}-01-01".format(y)) & (df['date'] < "{}-01-08".format(y))].head(1)
        jan_sell = df[ (df['date'] >= "{}-01-01".format(y)) & (df['date'] < "{}-01-08".format(y))].tail(1)
        jan_profit = jan_purchase['open'].values - jan_sell['close'].values
                
        dec_purchase = df[ (df['date'] >= "{}-12-01".format(y-1)) & (df['date'] < "{}-12-08".format(y))].head(1)
        dec_sell = df[ (df['date'] >= "{}-12-01".format(y-1)) & (df['date'] < "{}-12-08".format(y))].tail(1)
        dec_profit = dec_purchase['open'].values - dec_sell['close'].values
        
        comp =  jan_profit > dec_profit
        score = comp.sum()
        profit_1w_result[c] += score
    profit_1w_result[c] /= len(yrs) * 1.0

In [73]:
profit_1w_p = sum(profit_1w_result.values())/ len(comps)*100.0
profit_1w_p

71.03960396039604

How do we know this is not due to the change of month?

In [88]:
change_of_month_result = {}
months = range(2, 12) # already examed jan
for c in comps:
    df = data[data['Name'] == c]
    change_of_month_result[c] = 0
    for y in yrs:
        for month in months: 
            m = str(month).zfill(2)
            m1 = str(month-1).zfill(2)
            curr_purchase = df[(df['date'] >= "{0}-{1}-01".format(y, m)) & (df['date'] < "{0}-{1}-08".format(y, m))].head(1)
            curr_sell = df[(df['date'] >= "{0}-{1}-01".format(y, m)) & (df['date'] < "{0}-{1}-08".format(y, m))].tail(1)
            curr_profit = curr_purchase['open'].values - curr_sell['close'].values
                
            prev_purchase = df[(df['date'] >= "{0}-{1}-01".format(y, m1)) & (df['date'] < "{0}-{1}-08".format(y, m1))].head(1)
            prev_sell = df[(df['date'] >= "{0}-{1}-01".format(y, m1)) & (df['date'] < "{0}-{1}-08".format(y, m1))].tail(1)
            prev_profit = prev_purchase['open'].values - prev_sell['close'].values
        
            comp =  curr_profit > prev_profit
            score = comp.sum()
            change_of_month_result[c] += score
    change_of_month_result[c] /= len(months) * len(yrs) * 1.0


In [91]:
change_of_month_p = 1.0 * sum(change_of_month_result.values())/len(comps)* 100
change_of_month_p

47.64851485148508